## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-28-01-14-00 +0000,wsj,U.S. Politics and Policy Bias Favor Weaker Dollar,https://www.wsj.com/finance/currencies/yen-wea...
1,2026-01-28-01-13-30 +0000,nyt,Initial D.H.S. Review Does Not Say Pretti Bran...,https://www.nytimes.com/2026/01/27/us/politics...
2,2026-01-28-01-10-07 +0000,nypost,Doc fumes as Santa Monica launches woke progra...,https://nypost.com/2026/01/27/us-news/doc-fume...
3,2026-01-28-01-08-31 +0000,nyt,One Person Is Wounded in Shooting Involving U....,https://www.nytimes.com/2026/01/27/us/border-s...
4,2026-01-28-01-05-00 +0000,wsj,FAA Failures Created Conditions for Deadly Rea...,https://www.wsj.com/business/airlines/faa-fail...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2432/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
54,trump,69
279,ice,22
135,minnesota,20
170,year,18
113,minneapolis,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
23,2026-01-28-00-24-00 +0000,wsj,President Trump defended Homeland Security Sec...,https://www.wsj.com/politics/policy/rump-krist...,170
327,2026-01-27-10-28-15 +0000,cbc,Judge in Minnesota orders acting ICE director ...,https://www.cbc.ca/news/world/bovino-ice-minne...,146
302,2026-01-27-12-02-00 +0000,wsj,Less than 48 hours after U.S. agents fatally s...,https://www.wsj.com/politics/policy/homan-in-b...,145
273,2026-01-27-14-15-00 +0000,wsj,President Trump’s pivot in Minneapolis came af...,https://www.wsj.com/politics/policy/the-48-hou...,144
363,2026-01-27-03-01-00 +0000,wsj,"Democratic Rep. Ilhan Omar of Minnesota, facin...",https://www.wsj.com/politics/policy/minnesota-...,143


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
23,170,2026-01-28-00-24-00 +0000,wsj,President Trump defended Homeland Security Sec...,https://www.wsj.com/politics/policy/rump-krist...
205,75,2026-01-27-17-53-30 +0000,nypost,TikTok’s new owners accused of censoring Minne...,https://nypost.com/2026/01/27/business/tiktok-...
241,58,2026-01-27-15-59-40 +0000,nypost,Meta and YouTube face landmark trial over yout...,https://nypost.com/2026/01/27/business/meta-ti...
302,56,2026-01-27-12-02-00 +0000,wsj,Less than 48 hours after U.S. agents fatally s...,https://www.wsj.com/politics/policy/homan-in-b...
285,53,2026-01-27-13-22-00 +0000,wsj,EU and India Reach Free-Trade Deal as World Re...,https://www.wsj.com/economy/trade/eu-and-india...
183,48,2026-01-27-18-41-00 +0000,wsj,Eleanor Holmes Norton said she won’t run for r...,https://www.wsj.com/politics/eleanor-holmes-no...
12,48,2026-01-28-00-46-53 +0000,bbc,'We all know someone who was killed' - Iran p...,https://www.bbc.com/news/articles/cy8y2jxx9ppo...
361,43,2026-01-27-03-36-00 +0000,wsj,"Deadly Winter Storm Disrupts Air Travel, Leave...",https://www.wsj.com/us-news/climate-environmen...
72,43,2026-01-27-22-59-24 +0000,nyt,Bari Weiss Urges CBS News to Think Like a ‘Sta...,https://www.nytimes.com/2026/01/27/business/me...
39,39,2026-01-27-23-53-27 +0000,nypost,Citi slams sexual harassment claims against to...,https://nypost.com/2026/01/27/business/citigro...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
